In [1]:
f1 = open("train.txt","r")
raw_data = f1.readlines()
f1.close()

In [2]:
from collections import defaultdict
node_set = set()
in_neighbour = defaultdict(set)  
out_neighbour = defaultdict(set) 
for instance in raw_data:  
    source,sink_list = instance.split()[0],instance.split()[1:]
    node_set.add(source)
    out_neighbour[source] = set(sink_list)
    for node in sink_list:
        node_set.add(node)
        in_neighbour[node].add(source)
#print (in_neighbour)
print (len(node_set))

4867136


In [3]:
sum_degree = 0
for node in node_set:
    degree = len(in_neighbour[node])+len(out_neighbour[node])
    sum_degree += degree
print (sum_degree)

47893204


In [49]:
import math    
def common_neighbour(node1,node2):
    return len(in_neighbour[node1].intersection(in_neighbour[node2]))+len(out_neighbour[node1].intersection(out_neighbour[node2]))

def salton(node1,node2):
    return common_neighbour(node1,node2)/math.sqrt((len(in_neighbour[node1])+len(out_neighbour[node1]))*(len(in_neighbour[node2])+len(out_neighbour[node2])))

def allocation(node1,node2):
    result = 0
    common1 = in_neighbour[node1].intersection(in_neighbour[node2]).union(out_neighbour[node1].intersection(out_neighbour[node2]))
    common2 = in_neighbour[node1].intersection(out_neighbour[node2]).union(out_neighbour[node1].intersection(in_neighbour[node2]))
    common = common1.union(common2)        
    for node in common:
        result += 1/(len(out_neighbour[node])+len(in_neighbour[node]))
    return result

def AA(node1,node2):
    result = 0
    intersection = in_neighbour[node1].intersection(in_neighbour[node2]).union(out_neighbour[node1].intersection(out_neighbour[node2]))
    for node in intersection:
        result += (1/math.log(len(in_neighbour[node])+len(out_neighbour[node])))
    return result


def knn(node1,node2):
    result=[]
    firts=1
    Wbin=1/math.sqrt(1+len(in_neighbour[node2]))
    Waout=1/math.sqrt(1+len(out_neighbour[node1]))
    w2=Waout
    w3=Wbin
    w4=Wbin+Waout
    w5=Wbin*Waout
    result.append(w2)
    result.append(w3)
    result.append(w4)
    result.append(w5)
    return result
def Jaccard(node1,node2):
    return common_neighbour(node1,node2)/(len(in_neighbour[node1].union(in_neighbour[node2]))+len(out_neighbour[node1].union(out_neighbour[node2])))
def CNPA(node1,node2):
    result = 0
    sigma = 0.0000000001
    PA = (len(in_neighbour[node1])+len(out_neighbour[node1]))*(len(in_neighbour[node2])+len(out_neighbour[node2]))
    return common_neighbour(node1,node2)+(sigma*PA)/(sum_degree/len(node_set))

def get_feature(node1,node2):
    result=[]
    result.append(salton(node1,node2))
    result.append(allocation(node1,node2))
    result.append(Jaccard(node1,node2))
    result.extend(knn(node1,node2))
    #result.append(AA(node1,node2))
    result.append(CNPA(node1,node2))
    return result

In [6]:
f=open("train.txt","r")
number=[]
L1=[]
for k in f:
    a=k.strip()
    y=a.split()
    for i in y:
        number.append(i)
    L1.append(number)
    number=[]

In [7]:
x_list=[]
y_list=[]
one_hot_x=[]
one_hot_y=[]
count=0
for i in L1:
    x_list.append(i[0])
    y_list.append(i[1:])

In [43]:
import random
positive_list = []
new_source_list = random.sample(x_list,300)

In [44]:
positive_list =set()
Tlist=[]
for i in new_source_list:
    Tlist.append(y_list[x_list.index(i)])    

In [45]:
target_list=[]
for i in Tlist:
    for k in i:
        target_list.append(k) 

In [46]:
for i in range(len(new_source_list)):
    for k in Tlist[i]:
        positive_list.add((new_source_list[i],k))

In [47]:
print(len(positive_list))

228179


In [48]:
import random
negative_list=set()
count=0
for i in new_source_list:
    for k in target_list:
        if (i,k) not in positive_list:
            negative_list.add((i,k))
    count=count+1
    if count%30==0:
        print(count)
negative_list2=random.sample(negative_list,100000)

30
60
90
120
150
180
210
240
270
300


In [34]:
print(len(negative_list2))

100000


In [64]:
training_set = []
count=0
for node1,node2 in positive_list:
    outcome = get_feature(node1,node2)
    outcome.append([1,0])
    training_set.append(outcome)
    count=count+1
    if count%5000==0:
        print(count)
for node1,node2 in negative_list2:
    outcome = get_feature(node1,node2)
    outcome.append([0,1])
    training_set.append(outcome)
print (len(training_set))  

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
328179


In [65]:
X_train = []
Y_train = []
for item in training_set:
    X_train.append(item[:len(item)-1])
    Y_train.append(item[-1])

In [19]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [50,100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [20,50]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6,8]
# Method of selecting samples for training each tree
#bootstrap = [True]
# Create the random grid
para_grid = {'n_estimators': n_estimators,
             #'max_features': max_features,
             #'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np


rf = RandomForestRegressor(bootstrap=True, criterion='mse',
           max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False )
grid_search = GridSearchCV(estimator = rf, param_grid = para_grid, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_search.fit(X_train, Y_train)
best_grid = grid_search.best_estimator_
print (best_grid)
print ("train success!")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=50 ........
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=50 ........
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=50 ........
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=100 .......
[CV]  min_samples_leaf=1, min_samples_split=2, n_estimators=50, total=   2.4s
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=100 .......
[CV]  min_samples_leaf=1, min_samples_split=2, n_estimators=50, total= 1.4min
[CV] min_samples_leaf=1, min_samples_split=2, n_estimators=100 .......
[CV]  min_samples_leaf=1, min_samples_split=2, n_estimators=50, total= 1.4min
[CV] min_samples_leaf=1, min_samples_split=3, n_estimators=50 ........
[CV]  min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   2.0s
[CV] min_samples_leaf=1, min_samples_split=3, n_estimators=50 ........
[CV]  min_samples_leaf=1, min_samples_split=2, n_estimato

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 24.3min


[CV]  min_samples_leaf=2, min_samples_split=2, n_estimators=100, total= 2.9min
[CV] min_samples_leaf=2, min_samples_split=2, n_estimators=100 .......
[CV] min_samples_leaf=2, min_samples_split=2, n_estimators=100 .......
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=50 ........
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=50 ........
[CV]  min_samples_leaf=2, min_samples_split=2, n_estimators=100, total=   2.4s
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=50 ........
[CV]  min_samples_leaf=2, min_samples_split=3, n_estimators=50, total=   1.6s
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=100 .......
[CV]  min_samples_leaf=2, min_samples_split=3, n_estimators=50, total= 1.3min
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=100 .......
[CV]  min_samples_leaf=2, min_samples_split=3, n_estimators=50, total= 1.3min
[CV] min_samples_leaf=2, min_samples_split=3, n_estimators=100 .......
[CV]  min_samples_leaf=2, min_samples_sp

[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=50 ........
[CV]  min_samples_leaf=4, min_samples_split=10, n_estimators=100, total=   1.9s
[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=50 ........
[CV]  min_samples_leaf=6, min_samples_split=2, n_estimators=50, total= 1.1min
[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=50 ........
[CV]  min_samples_leaf=6, min_samples_split=2, n_estimators=50, total= 1.1min
[CV]  min_samples_leaf=6, min_samples_split=2, n_estimators=50, total=   1.5s
[CV]  min_samples_leaf=4, min_samples_split=10, n_estimators=100, total= 2.2min
[CV]  min_samples_leaf=4, min_samples_split=10, n_estimators=100, total= 2.2min
[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=100 .......
[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=100 .......
[CV] min_samples_leaf=6, min_samples_split=2, n_estimators=100 .......
[CV]  min_samples_leaf=6, min_samples_split=2, n_estimators=100, total=   2.2s
[CV]  min_samples_lea

[CV]  min_samples_leaf=8, min_samples_split=8, n_estimators=100, total=   2.1s
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=50, total= 1.2min
[CV]  min_samples_leaf=8, min_samples_split=8, n_estimators=100, total= 2.2min
[CV] min_samples_leaf=8, min_samples_split=10, n_estimators=50 .......
[CV] min_samples_leaf=8, min_samples_split=10, n_estimators=50 .......
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=50, total=   1.4s
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=50, total= 1.0min
[CV] min_samples_leaf=8, min_samples_split=10, n_estimators=100 ......
[CV] min_samples_leaf=8, min_samples_split=10, n_estimators=100 ......
[CV] min_samples_leaf=8, min_samples_split=10, n_estimators=100 ......
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=100, total=   2.4s
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=100, total= 1.5min
[CV]  min_samples_leaf=8, min_samples_split=10, n_estimators=100, total= 1.5min


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 92.7min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
train success!


In [66]:
import csv
fp = open("test-public_1.txt","r")
head = fp.readline()
test_file = fp.readlines()
fp.close()
predict_list=[]
print (test_file[0])
test_data = []
for line in test_file:
    Id,node1,node2 = line.split()
    outcome = get_feature(node1,node2)
    test_data.append(outcome)

1	2184483	1300190

